In [2]:
using Images
using FileIO
using DynamicPolynomials

@polyvar x
@polyvar y
@polyvar z
@polyvar w



(w,)

In [3]:

function progs(ndims, order)
    m = 1
    dims = ["x", "y", "z", "w"]

    progs = Dict{String, Vector{String}}()
    for i in 1:ndims
        d = dims[i]
        progs[d] = Vector{String}()
        push!(progs[d], "a[$m]")
        m += 1
        for i1 in 1:ndims
            push!(progs[d], "a[$m] * $(dims[i1])")
            m += 1
            for i2 in i1:ndims
                push!(progs[d], "a[$m] * $(dims[i1]) * $(dims[i2])")
                m += 1
                if order > 2
                    for i3 in i2:ndims
                        push!(progs[d], "a[$m] * $(dims[i1]) * $(dims[i2]) * $(dims[i3])")
                        m += 1
                        if order > 3
                            for i4 in i3:ndims
                                push!(progs[d], "a[$m] * $(dims[i1]) * $(dims[i2]) * $(dims[i3]) * $(dims[i4])")
                                m += 1
                                if order > 4
                                    for i5 in i4:ndims
                                        push!(progs[d], "a[$m] * $(dims[i1]) * $(dims[i2]) * $(dims[i3]) * $(dims[i4]) * $(dims[i5])")
                                        m += 1
                                    end
                                end
                            end
                        end
                    end
                end
            end
        end
    end
    progs, m-1
end


progs (generic function with 1 method)

In [4]:
function polys(es::Dict{String, Expr})
    ps = Dict{String, DynamicPolynomials.Polynomial{true,Float64}}()

    for d in keys(es)
        ps[d] = eval(es[d])
    end
    ps
end

polys (generic function with 1 method)

In [5]:

function exprs(ps::Dict{String, Vector{String}})

    es = Dict{String, Expr}()
    for d in keys(ps)
        es[d] = Meta.parse(join(ps[d], " + "))
    end

    es
end


exprs (generic function with 1 method)

In [6]:


function string2args(s::String)
    [(Int(c)-77)/10.0 for c in s]
end


string2args (generic function with 1 method)

In [7]:


function write_raw(iters, w, h)
    hrange = (minimum(iters["x"]), maximum(iters["x"]))
    hscale = (w-1) / (hrange[2]-hrange[1])
    yrange = (minimum(iters["y"]), maximum(iters["y"]))
    yscale = (h-1) / (yrange[2]-yrange[1])

    pixels = zeros(Int64, w, h)

    for i in 1:length(iters["x"])
        x = floor(Int64, 1+(iters["x"][i] - hrange[1]) * hscale)
        y = floor(Int64, 1+(iters["y"][i] - yrange[1]) * yscale)
        pixels[x, y] += 1
    end

    gmax = 1 / maximum(pixels)
    
    gs =  (pixels .* gmax) |> colorview(Gray)
    
    save("t.jpg", gs)
    gs
end

write_raw (generic function with 1 method)

In [8]:
function iterate(ps, states, ns)
    
    n = 1 + ns[end]-ns[1]

    iters = Dict{String, Vector{Float64}}()
    for d in keys(ps)
        iters[d] = zeros(Float64, n)
    end


    for i in 1:n
        for d in keys(iters)
            states[d] = iters[d][i] = ps[d](x=>states["x"], y=>states["y"], z=>states["z"], w=>states["w"])
        end
    end

    iters
end

iterate (generic function with 1 method)

In [17]:
function lyup(vs)
    L = 0.
    for i in 2:length(vs)
        L += abs(vs[i-1] - vs[i])
    end
    L /= length(vs)
    0.721347 * log(L)
end

lyup (generic function with 1 method)

In [112]:
function valid(iters)
   for d in keys(iters)
        ld = lyup(iters[d])
        if isnan(ld) || ld == Inf
            return false
        end
    end
    return true
end

valid (generic function with 1 method)

In [130]:
using Random 

while true
    pgs, as = progs(2,2)
    a = rand(as)
    es = exprs(pgs)
    pl = polys(es)

    states = Dict{String, Float64}()
    states["x"] = 0.05
    states["y"] = 0.05
    states["z"] = 0.05
    states["w"] = 0.05

    iters = iterate(pl, states, 1:10)

    if valid(iters)
        println(iters)
        println(states)
        break
    end
end


Dict("x" => [0.22061995792071332, 0.3305425136851784, 0.44688653443115034, 0.618631161509061, 0.9770781273448367, 2.2589801939561562, 19.184856690831715, 30512.619313996274, 7.511635474886118e16, 2.6554310411979717e66], "y" => [0.08366812896268386, 0.13834209069734316, 0.2284103272940013, 0.4132513447996993, 0.9731809539534083, 4.66263258581606, 239.61747950669724, 3.810836794127402e8, 2.2658111676935248e33, 2.8315533874942928e132])
Dict("w" => 0.05, "x" => 2.6554310411979717e66, "z" => 0.05, "y" => 2.8315533874942928e132)


In [ ]:


println(pl)
dx = differentiate(pl["x"], x)
dy = differentiate(pl["y"], y)

for i in 1:100
    [dx(x=)]